<a href="https://colab.research.google.com/github/mrbisao/BOOTCAMP-DIO-AI102/blob/main/Desafio_Tradutor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# =====================  Imports =====================
from bs4 import BeautifulSoup
import requests, os, time, json
from dotenv import load_dotenv


load_dotenv("config.env", override=True)

AZURE_ENDPOINT   = os.getenv("AZURE_ENDPOINT", "").rstrip("/")
AZURE_API_KEY    = os.getenv("AZURE_OPENAI_KEY")
AZURE_DEPLOYMENT = os.getenv("AZURE_DEPLOYMENT", "o4-mini")
AZURE_API_VER    = os.getenv("AZURE_API_VERSION", "2025-01-01-preview")

CHAT_URL = f"{AZURE_ENDPOINT}/openai/deployments/{AZURE_DEPLOYMENT}/chat/completions?api-version={AZURE_API_VER}"
HEADERS  = {"Content-Type": "application/json", "api-key": AZURE_API_KEY}

print("CHAT_URL:", CHAT_URL)

# =====================  Extração de texto =====================
def extract_text(url: str) -> str | None:
    r = requests.get(url, timeout=40)
    if r.status_code != 200:
        print("Falha ao buscar a URL. Código:", r.status_code); return None
    soup = BeautifulSoup(r.text, "html.parser")
    for tag in soup(["script","style","noscript"]): tag.decompose()
    for sel in ("div.crayons-article__body", "div#article-body", "article"):
        node = soup.select_one(sel)
        if node and node.get_text(strip=True):
            return node.get_text(" ", strip=True)
    return soup.get_text(" ", strip=True)

# =====================  Utilitários =====================
def remove_codefences(text: str) -> str:
    if not text: return text
    t = text.strip()
    if t.startswith("```"):
        lines = t.splitlines()
        if lines and lines[0].startswith("```"): lines = lines[1:]
        if lines and lines[-1].strip() == "```": lines = lines[:-1]
        t = "\n".join(lines).strip()
    return t

def call_chat_completions(text_chunk: str, lang: str, max_tokens: int):
    # Prompt enxuto, pedindo para NÃO usar cercas de código e sem explicações
    messages = [
        {"role":"system","content":"Você é um tradutor profissional. Faça tradução direta e fiel. Não adicione comentários, nem explicações. Não use blocos de código (sem ```). Responda apenas com a tradução em Markdown simples."},
        {"role":"user","content":f"Traduza para {lang} o texto abaixo, mantendo títulos e listas quando fizer sentido:\n\n{text_chunk}"}
    ]
    payload = {
        "messages": messages,
        "max_completion_tokens": max_tokens,
        "temperature": 1  # se seu deployment não aceitar, removemos no retry abaixo
    }

    r = requests.post(CHAT_URL, headers=HEADERS, json=payload, timeout=180)
    try:
        data = r.json()
    except Exception:
        raise SystemExit(f"Status={r.status_code}\nResposta bruta:\n{r.text}")

    # retry automático se algum parâmetro for rejeitado
    if r.status_code == 400 and isinstance(data, dict):
        err = (data.get("error") or {})
        if err.get("code") == "unsupported_parameter":
            param = err.get("param")
            if param in payload:
                payload.pop(param, None)
                r = requests.post(CHAT_URL, headers=HEADERS, json=payload, timeout=180)
                try:
                    data = r.json()
                except Exception:
                    raise SystemExit(f"[Retry] Status={r.status_code}\nResposta bruta:\n{r.text}")

    if r.status_code != 200:
        raise SystemExit(f"Erro {r.status_code}:\n{json.dumps(data, ensure_ascii=False, indent=2)}")

    choice = (data.get("choices") or [{}])[0]
    content = ((choice.get("message") or {}).get("content") or "").strip()
    finish  = choice.get("finish_reason")
    usage   = data.get("usage", {})
    return remove_codefences(content), finish, usage

def translate_chunk_adaptativo(text_chunk: str, lang: str) -> str:
    # 1) tenta com tokens maiores; se vier vazio/length, divide
    for tok in (2048, 4096, 6144):
        out, finish, usage = call_chat_completions(text_chunk, lang, tok)
        if out:
            return out
        if finish == "length":
            print(f"[chunk vazio] finish_reason=length usage={usage}")
            continue
    # 2) dividir em 2 sub-blocos se ainda vazio
    if len(text_chunk) > 300:
        mid = len(text_chunk)//2
        left  = translate_chunk_adaptativo(text_chunk[:mid], lang)
        right = translate_chunk_adaptativo(text_chunk[mid:], lang)
        return (left + "\n\n" + right).strip()
    # 3) último esforço com tokens menores (às vezes resolve)
    out, _, _ = call_chat_completions(text_chunk, lang, 1024)
    return out

# ===================== 5) Tradução com chunking =====================
def translate_article(full_text: str, lang: str, chunk_chars: int = 800) -> str:
    if not full_text: return ""
    parts = [full_text[i:i+chunk_chars] for i in range(0, len(full_text), chunk_chars)]
    translated = []
    for i, chunk in enumerate(parts, 1):
        print(f"Traduzindo bloco {i}/{len(parts)} ({len(chunk)} chars)...")
        t = translate_chunk_adaptativo(chunk, lang)
        translated.append(t or "")
        time.sleep(0.2)
    return "\n\n".join(translated).strip()

# ===================== 6) Uso =====================
# sanity check
print("Teste curto:", translate_chunk_adaptativo("Hello world", "português")[:120])

# Artigo OSI
url = "https://dev.to/harshm03/network-layer-internet-protocol-computer-networks-4847"
text = extract_text(url)
if text:
    artigo_pt = translate_article(text, "português", chunk_chars=800)
    print("\n--- Início da tradução ---\n")
    print(artigo_pt[:1500])

    # salvar em arquivo .md
    with open("artigo_traduzido.md", "w", encoding="utf-8") as f:
        f.write(artigo_pt)
    print("\nArquivo salvo: artigo_traduzido.md")
else:
    print("Não foi possível extrair texto da URL.")


CHAT_URL: https://oai-dio-bootcamp-dev-eastuus-mrbisao.openai.azure.com/openai/deployments/o4-mini/chat/completions?api-version=2025-01-01-preview
Teste curto: Olá mundo
Traduzindo bloco 1/33 (800 chars)...
Traduzindo bloco 2/33 (800 chars)...
Traduzindo bloco 3/33 (800 chars)...
[chunk vazio] finish_reason=length usage={'completion_tokens': 2048, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 2048, 'rejected_prediction_tokens': 0}, 'prompt_tokens': 219, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}, 'total_tokens': 2267}
Traduzindo bloco 4/33 (800 chars)...
Traduzindo bloco 5/33 (800 chars)...
[chunk vazio] finish_reason=length usage={'completion_tokens': 2048, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 2048, 'rejected_prediction_tokens': 0}, 'prompt_tokens': 215, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}, 'total_tokens': 2263}
Tra